In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
print(os.listdir('drive'))

# import sys
# sys.path.append('drive/places_fc2')
# sys.path.append('drive/cnn')

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
['.Trash', 'open-flask', 'facial', 'control ', 'control 1', 'Document from Sandipan Dutta', 'dbms2', 'Document from Sandipan Dutta (b9c87a83)', 'bio', 'xray', 'gan', 'Copy of Interspeech.ipynb', 'siamese', 'tfsiamese', 'capgen', 'data', 'imgcap', 'Colab Notebooks', 'GupShup', 'Music Recommender', 'Distilled AI

In [0]:
#script for preparing datasets, loading fer data and generating scaled images 
import pandas as pd
import numpy as np

#make a directory, keep the fer2013.csv file in the directory and also the git in the same. Otherwise, change the directory in the scripts
#load fer2013.csv

data = pd.read_csv('drive/facial/fer2013.csv')
data = data['pixels']
data = [ dat.split() for dat in data]
data = np.array(data)
data = data.astype('float64')
data = [[np.divide(d,255.0) for d in dat] for dat in data]

np.save('drive/facial/data/Scaled.bin.npy',data)


In [0]:
import numpy as np
import pandas as pd
import cv2
data = pd.read_csv('drive/facial/fer2013.csv')
pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), (64,64))
    faces.append(face.astype('float32'))
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = pd.get_dummies(data['emotion']).as_matrix()


In [3]:
import numpy as np
import pandas as pd
np.save('drive/facial/face.npy',face)
np.save('drive/facial/emotions.npy',emotions)

NameError: ignored

In [0]:
emotions.shape

(35887, 7)

In [9]:
!pip install keras
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop



Using TensorFlow backend.


In [0]:
np.random.seed(2222)  # for reproducibility

#Load the scaled data, both pixels and labels
X_train = np.load('drive/facial/data/Scaled.bin.npy')
Y = np.load('drive/facial/data/labels.bin.npy')


In [16]:
X_train.shape

(35887, 2304)

In [17]:
Y.shape

(35887,)

In [0]:
for i in range(35887):
  if Y[i]==0:
    Y[i]=0
  elif Y[i]==1:
    Y[i]=0
  elif Y[i]==2:
    Y[i]=0
  elif Y[i]==3:
    Y[i]=1
  elif Y[i]==4:
    Y[i]=0
  elif Y[i]==5:
    Y[i]=1
  elif Y[i]==6:
    Y[i]=1

In [0]:
#reshape the given pixels into 48 X 48 images
shapex , shapey = 48, 48
X_train = X_train.reshape(X_train.shape[0] ,  shapex , shapey,1)

#convert labels to one-hot-encoding
Y = np_utils.to_categorical(Y)


In [20]:
Y.shape

(35887, 2)

In [0]:
"""
Best performing model till now. Added layers to the webcamemocognizer one.
"""


model = Sequential()
model.add(Convolution2D(32, (3, 3), padding='valid', input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64,(3,3),padding='valid',activation='relu'))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128,(3,3),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256,(3,3),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128,kernel_initializer="lecun_uniform"))
#model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))


In [0]:
"""
From Blog.
"""

from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
model = Sequential()
 
#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu',padding='valid', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
 
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu',padding='valid'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='valid'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
 
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu',padding='valid'))
model.add(Conv2D(128, (3, 3), activation='relu',padding='valid'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(Flatten())
 
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
 
model.add(Dense(2, activation='softmax'))

In [25]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 128)         147584    
__________

In [0]:
"""
Deeper Networks
"""

from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2

def simple_CNN(input_shape, num_classes):

    model = Sequential()
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same',
                            name='image_array', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=256, kernel_size=(3, 3), padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(filters=num_classes, kernel_size=(3, 3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax',name='predictions'))
    return model

def big_XCEPTION(input_shape, num_classes):
  img_input = Input(input_shape)
  x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
  x = BatchNormalization(name='block1_conv1_bn')(x)
  x = Activation('relu', name='block1_conv1_act')(x)
  x = Conv2D(64, (3, 3), use_bias=False)(x)
  x = BatchNormalization(name='block1_conv2_bn')(x)
  x = Activation('relu', name='block1_conv2_act')(x)

  residual = Conv2D(128, (1, 1), strides=(2, 2),
                    padding='same', use_bias=False)(x)
  residual = BatchNormalization()(residual)

  x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
  x = BatchNormalization(name='block2_sepconv1_bn')(x)
  x = Activation('relu', name='block2_sepconv2_act')(x)
  x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
  x = BatchNormalization(name='block2_sepconv2_bn')(x)

  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual])

  residual = Conv2D(256, (1, 1), strides=(2, 2),
                    padding='same', use_bias=False)(x)
  residual = BatchNormalization()(residual)

  x = Activation('relu', name='block3_sepconv1_act')(x)
  x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
  x = BatchNormalization(name='block3_sepconv1_bn')(x)
  x = Activation('relu', name='block3_sepconv2_act')(x)
  x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
  x = BatchNormalization(name='block3_sepconv2_bn')(x)

  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual])
  x = Conv2D(num_classes, (3, 3),
          #kernel_regularizer=regularization,
          padding='same')(x)
  x = GlobalAveragePooling2D()(x)
  output = Activation('softmax',name='predictions')(x)

  model = Model(img_input, output)
  return model


def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model


In [38]:
#input_shape = (48, 48, 1)
num_classes = 2
#model = tiny_XCEPTION(input_shape, num_classes)
#model.summary()
#model = mini_XCEPTION(input_shape, num_classes)
#model.summary()
#model = big_XCEPTION(input_shape, num_classes)
#model.summary()
model = simple_CNN((48, 48, 1), num_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 48, 48, 16)        800       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 48, 48, 16)        12560     
_________________________________________________________________
activation_7 (Activation)    (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 24, 24, 32)        12832     
__________

In [0]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


In [0]:
from keras.optimizers import Adam
optimizer =Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)


In [0]:
model.compile(loss='binary_crossentropy', optimizer=sgd)


In [0]:
model.load_weights('drive/facial/deep_model_weights_normal.h5py')

In [32]:
model.fit(X_train,Y , batch_size=32 , epochs=10)


Epoch 1/10
35887/35887 [==============================] - 16s 432us/step - loss: 0.0254
Epoch 2/10
35887/35887 [==============================] - 15s 432us/step - loss: 0.0213
Epoch 3/10
14624/35887 [===========>..................] - ETA: 9s - loss: 0.0222

35887/35887 [==============================] - 15s 430us/step - loss: 0.0268
Epoch 4/10
35887/35887 [==============================] - 15s 430us/step - loss: 0.0260
Epoch 5/10
22464/35887 [=================>............] - ETA: 5s - loss: 0.0237

35887/35887 [==============================] - 15s 429us/step - loss: 0.0256
Epoch 6/10
35887/35887 [==============================] - 15s 428us/step - loss: 0.0242
Epoch 7/10
24832/35887 [===================>..........] - ETA: 4s - loss: 0.0208

35887/35887 [==============================] - 15s 431us/step - loss: 0.0231
Epoch 8/10
35887/35887 [==============================] - 15s 431us/step - loss: 0.0225
Epoch 9/10
25440/35887 [====================>.........] - ETA: 4s - loss: 0.0210

35887/35887 [==============================] - 15s 429us/step - loss: 0.0218
Epoch 10/10
35887/35887 [==============================] - 15s 431us/step - loss: 0.0197


In [0]:
import h5py
#json_string = model.to_json()
model.save_weights('drive/facial/deep_model_weights_biinary_best.h5py')
#open('drive/facial/models/Face_model_architecture.json', 'w').write(json_string)
#model.save_weights('drive/facial/models/Face_model_weights.h5')
